# Let's chat with a friend and I have a brain

Demo chat with Leolani that combine face recognition and storage in the BRAIN. This notebook combines the functions of two other notebooks:


In [3]:
# general imports for EMISSOR and the BRAIN
import emissor as em
from cltl import brain
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario
from cltl.brain.long_term_memory import LongTermMemory
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from cltl.brain.utils.helper_functions import brain_response_to_json


# specific imports
import uuid
from datetime import datetime
import time
import cv2
import requests
import pickle


/Users/piek/PycharmProjects/cltl-chatbots/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk
import chatbots.util.face_util as f_util
import chatbots.intentions.get_to_know_you as friend

### Connect to the camera

In [5]:
### Link your camera
camera = cv2.VideoCapture(0)

### Initialise the BRAIN

In [6]:
# Initialise the brain in GraphDB
import pathlib

log_path = pathlib.Path('./logs')
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                                log_dir=log_path,
                                clear_all=True)

2021-11-15 16:18:14,433 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Booted


Booted


2021-11-15 16:18:19,712 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain


Uploading ontology to brain


2021-11-15 16:18:22,361 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Booted


...

Booted


2021-11-15 16:18:22,368 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Booted


Booted


2021-11-15 16:18:22,669 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


Computed trust for all known agents


## Create an instance of a replier

In [7]:
replier = LenkaReplier()

2021-11-15 16:18:32,518 -     INFO -   cltl.reply_generation.api.LenkaReplier - Booted


Booted


## Standard initialisation of a scenario

In [8]:
from random import getrandbits, choice
import requests
##### Setting the location
place_id = getrandbits(8)
location = None
try:
    location = requests.get("https://ipinfo.io").json()
except:
    print("failed to get the IP location")
    
##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
# Find the repository root dir
parent, dir_name = (d_util.__file__, "_")
while dir_name and dir_name != "src":
    parent, dir_name = os.path.split(parent)
root_dir = parent
scenario_path = os.path.abspath(os.path.join(root_dir, 'data'))

if scenario_path not in sys.path:
    sys.path.append(scenario_path)

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)

### Specify the path to an existing folder with the embeddings of your friends
friends_path = os.path.abspath(os.path.join(root_dir, 'friend_embeddings'))
if friends_path not in sys.path:
    sys.path.append(friends_path)
    print("The paths with the friends:", friends_path)
    
### Define the folders where the images and rdf triples are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"


### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario_ctrl = scenarioStorage.create_scenario(scenario_id, int(time.time() * 1e3), None, AGENT)

The paths with the friends: /Users/piek/PycharmProjects/cltl-chatbots/friend_embeddings
Directories for 2021-11-15-16:18:33 created in /Users/piek/PycharmProjects/cltl-chatbots/data


## Loading the docker containers for face detection and face property detection

#### If the docker images are running you can skip the next part

In [9]:
### This is only needed if you start the docker containers from this notebook

#container_fdr = f_util.start_docker_container("tae898/face-detection-recognition:v0.1", 10002)
#container_ag = f_util.start_docker_container("tae898/age-gender:v0.2", 10003)
#container_yolo = f_util.start_docker_container("tae898/yolov5", 10004)

If there is a problem starting the dockers, you may need to kill them and start them again. Use the following command to kill and rerun the previous command. Note that if there are running already you should not restart. Starting it again gives an error that the port is occupied.

In [10]:
#!docker kill $(docker ps -q)

## Extracting and storing triples from an utterance

The next function processes any textSignal using the baseline language model from the Leolani platform.
This model uses a context-free grammar and closed-class lexicons specifically designed for social robot interaction.

The function creates a capsule for posting any triples and perspective to the BRAIN.
If no triples are extracted, a dummy prompt is returned: "Any gossip?".

Any thoughts coming from the BRAIN are returned.

## Create a new friend.

The functions in *intentions/get_to_know_you.py* are needed to get the properties and visual information for identifying a new friend.

The visual information is based on the camera images of the uses from which we extract an averaged embedding.
These embeddings are store in the *friend_embeddings* folder. 

By comparing an image with the stored embeddings, the system decides whether a person is a *stranger*.
In case the user is a *stranger*, the system will try to get to know him/her.

If you delete someone's embeddings from the *friend_embeddings* folder. This person will become a *stranger* again.

In [11]:
def parse_age(face_info):
    return round(face_info.age["mean"])
def parse_gender(face_info):
    return "male" if face_info.gender["m"] > 0.5 else "female"
def parse_bbox(face_info):
    return [int(num) for num in face_info.bbox.tolist()]
def parse_id(face_info):
    return face_info.face_id['name'] if 'name' in face_info.face_id else f"Stranger_t_{int(time.time() * 1e3)}"
def parse_name(face_info):
    face_id = parse_id(face_info)
    return face_id.split("_t_")[0] if face_id else "Stranger"

# First signals to get started
faces =[]
while not len(faces) == 1:
    success, frame = camera.read()
    if not success:
        raise ValueError("Failed to take a picture")
        
    image_time = int(time.time() * 1e3)
    imagepath = d_util.absolute_path(scenarioStorage, scenario_id, Modality.IMAGE, f"{image_time}.png")
    cv2.imwrite(imagepath, frame)
    
    faces = f_util.detect_faces(friends_path, imagepath)
    
    image_bbox = (0, 0, frame.shape[1], frame.shape[0])
    imageSignal = d_util.create_image_signal(scenario_ctrl, f"{image_time}.png", image_bbox, image_time)
    mentions = [f_util.create_face_mention(imageSignal, "front_camera", image_time,
                                           parse_bbox(face), parse_id(face), parse_name(face),
                                           parse_age(face), parse_gender(face), face.det_score)
                for face in faces]
    imageSignal.mentions.extend(mentions)
    scenario_ctrl.append_signal(imageSignal)

    if not faces:
        response = "Hi, anyone there? I can't see you.."
        time.sleep(3)
    elif len(faces) > 1:
        response = "Hi there! Apologizes, but I will only talk to one of you at a time.."
        time.sleep(3)
    else:
        face = faces[0]
        if parse_id(face) is None:
            ### This is a stranger, we process the new face
            HUMAN_ID, HUMAN_NAME, _ = friend.get_to_know_person(scenario_ctrl, AGENT, parse_gender(face),
                                                                parse_age(face), face.face_id, face.embedding,
                                                                friends_path)
            HUMAN_ID = HUMAN_NAME  ### Hack because we cannot force the namespace through capsules, name and identity are the same till this is fixed


            ### Add the new information to the signal
            mention = f_util.create_face_mention(imageSignal, "front_camera", image_time,
                                                 parse_bbox(face), human_id, human_name,
                                                 parse_age(face), parse_gender(face), face.det_score)
            imageSignal.mentions.append(mention)

            response = f"So you what do you want to talk about {human_name}?"
        else:
            ### We know this person
            HUMAN_ID = parse_id(face)
            HUMAN_NAME = parse_name(face)
            response = f"Hi {parse_name(face)}. Nice to see you again. How are you today?"

    print(f"{AGENT}: {response}\n")

    # Store signals, annotated with the infered Person information
    textSignal = d_util.create_text_signal(scenario_ctrl, response)
    scenario_ctrl.append_signal(textSignal)
    
scenarioStorage.save_scenario(scenario_ctrl)

/Users/piek/PycharmProjects/cltl-chatbots/data/2021-11-15-16:18:33/image/1636989523340.png image loaded!
got <Response [200]> from server!...
1 faces deteced!
got <Response [200]> from server!...
got <Response [200]> from server!...
YOLO image annotation is done!
Annotating face, genders, and ages is done!
image annotation is done!
image saved at /Users/piek/PycharmProjects/cltl-chatbots/data/2021-11-15-16:18:33/image/1636989523340.png.ANNOTATED.jpg


Leolani2: Hi Piek. Nice to see you again. How are you today?



## Proceed to communicate with an identified friend

The next while loop is for continuous communication with the identified user until *stop* or *bye*.
We process the user input and the captured image sequentially step by setp in the code block of the while loop.
At the end of the code block, a response is generated and new input from the user is requested.

In [13]:
chat = Chat(HUMAN_ID)

2021-11-15 16:20:17,467 -     INFO - cltl.triple_extraction.api.Chat (Piek)              000 - << Start of Chat with Piek >>


<< Start of Chat with Piek >>


We define a function that does the following:

* captures an image from the camera
* creates an imageSignal from it, 
* annotates the image with the mention of the person
* stores the mention in the brain as a perceivedBy data
* gets the response from the brain
* verbalises the response
* stores the verbalised response as a textSignal


In [15]:
def respond_to_your_face(camera, imagefolder, friends_path, scenario_ctrl, place_id, location, HUMAN_ID, my_brain, replier):
        ## We capture the image again
    ## For now, we only take pictures of faces of the user
    ## @TODO add object recognition and check if there are other people detected than the user

    success, frame = camera.read()
    if success:
        current_time = str(datetime.now().microsecond)
        imagepath = imagefolder + "/" + current_time + ".png"
        cv2.imwrite(imagepath, frame)
        faces = f_util.detect_faces(friends_path, imagepath)

        image_bbox = (0, 0, frame.shape[1], frame.shape[0])
        imageSignal = d_util.create_image_signal(scenario_ctrl, imagepath, image_bbox)
        container_id = str(uuid.uuid4())

        #### Properties are now stored as annotations
        #### We do not store these proeprties again to the BRAIN
        # Here we assume that only one face is in the image
        # TODO: deal with multiple people.
        if len(faces) != 1:
            raise ValueError("Only single face is supported, detected " + len(faces))

        face = faces[0]
        age = round(face.age["mean"])
        gender = "male" if face.gender["m"] > 0.5 else "female"
        bbox = [int(num) for num in face.bbox.tolist()]
        uuid_name = face.face_id
        faceprob = face.det_score
        embedding = face.embedding

        mention = f_util.create_face_mention(imageSignal, "front_camera", current_time,
                                   bbox, HUMAN_ID, HUMAN_NAME, age, gender, faceprob)
        imageSignal.mentions.append(mention)
        scenario_ctrl.append_signal(imageSignal)

        ### We created a perceivedBy triple for this experience, 
        ### @TODO we need to include the bouding box somehow in the object
        capsule = c_util.scenario_image_triple_to_capsule(scenario_ctrl,
                                                          place_id,
                                                          location,
                                                          imageSignal,
                                                          "front_camera",
                                                          HUMAN_ID,
                                                          "perceivedBy",
                                                          imageSignal.id)

        # Create the response from the system and store this as a new signal
        # We use the throughts to respond
        response = my_brain.update(capsule, reason_types=True, create_label=True)
        response_json = brain_response_to_json(response)
        reply = replier.reply_to_statement(response_json, proactive=True, persist=True)
        print(AGENT + ": " + reply)
        textSignal = d_util.create_text_signal(scenario_ctrl, reply)
        scenario_ctrl.append_signal(textSignal)


In [30]:
print_details=False
#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = f"{choice(TALK_TO_ME)}"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario_ctrl, initial_prompt)
scenario_ctrl.append_signal(textSignal)

utterance = ""
utterance = input('\n')
print(HUMAN_NAME + ": " + utterance)

#### Get input and loop
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    ###### Getting the next input signals
    textSignal = d_util.create_text_signal(scenario_ctrl, utterance)
    scenario_ctrl.append_signal(textSignal)

    #### Process input and generate reply
    
    capsule, reply = talk.process_text_and_reply(scenario_ctrl,
                           place_id,
                           location,
                           HUMAN_ID,
                           textSignal,
                           chat,
                           replier,
                           my_brain,
                           print_details)
          
    print(AGENT + ": " + reply)
    textSignal = d_util.create_text_signal(scenario_ctrl, reply)
    scenario_ctrl.append_signal(textSignal)

    ## We capture the image again and respond to it
    respond_to_your_face(camera, imagefolder, friends_path, scenario_ctrl, place_id, location, HUMAN_ID, my_brain, replier)
 
    # Get the next input signal from the user
    utterance = input("\n")
    print(HUMAN_NAME + ": " + utterance)


    ### We now have a new input check if the user wants to continue

Leolani2: Tell me anything, I love learning things



 I love pizza


Piek: I love pizza
2021-11-15 17:02:08,516 -     INFO - cltl.triple_extraction.api.Chat (Piek)              007 -       Piek: "I love pizza"


      Piek: "I love pizza"
extracted perspective: {'sentiment': '1', 'certainty': 1, 'polarity': 1, 'emotion': <Emotion.NEUTRAL: 7>}
RDF    subject: {"label": "Piek", "type": ["agent"]}
RDF  predicate: {"label": "love", "type": ["verb.emotion"]}
RDF     object: {"label": "pizza", "type": ["noun.food"]}


2021-11-15 17:02:08,643 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: piek_love_pizza [agent_->_food])


...

2021-11-15 17:02:54,241 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. experience touch - 15 gaps as object: e.g. cook-by dish


Gaps: 26 gaps as subject: e.g. experience touch - 15 gaps as object: e.g. cook-by dish


Leolani2: I would like to know. Has piek ever like by a interest?



 stop


Piek: stop


### Set the end time of the scenario, save it and stop the containers

After we stopped the interaction, we set the end time and save the scenario as EMISSOR data.

In [31]:
scenario_ctrl.scenario.ruler.end = int(time.time() * 1e3)
scenarioStorage.save_scenario(scenario_ctrl)

In [32]:
### Stopping the docker containers
### This is only needed of you started them in this notebook
#f_util.kill_container(container_fdr)
#f_util.kill_container(container_ag)

In [33]:
#### Stop the camera when we are done
camera.release()

## End of notebook